In [1]:
import tokenize
from io import BytesIO

def tokenize_code(code):
  tokens = []
  for tok in tokenize.tokenize(BytesIO(code.encode('utf-8')).readline):
    tokens.append({
      "type": tokenize.tok_name[tok.type],
      "string": tok.string
    })
  return tokens[1:]

lib_path = "D:/csharp_lib/MegaLib/MegaLib/lib/openxr_loader.dll"

In [4]:
function_list = [
  "XrResult xrCreateInstance(const XrInstanceCreateInfo* createInfo, XrInstance* instance)",
  
  "XrResult xrEnumerateApiLayerProperties(uint32_t propertyCapacityInput, uint32_t* propertyCountOutput, XrApiLayerProperties[] properties)",
  "XrResult xrEnumerateApiLayerProperties(uint32_t propertyCapacityInput, uint32_t* propertyCountOutput, IntPtr properties)",
  
  "XrResult xrDestroyInstance(XrInstance instance)",
  "XrResult xrGetInstanceProperties(XrInstance instance, XrInstanceProperties* instanceProperties)",
  "XrResult xrGetSystem(XrInstance instance, const XrSystemGetInfo* getInfo, XrSystemId* systemId)",
  "XrResult xrGetSystemProperties(XrInstance instance, XrSystemId systemId, XrSystemProperties* properties)",
  "XrResult xrCreateSession(XrInstance instance, const XrSessionCreateInfo* createInfo, XrSession* session)",
  #"XrResult xrGetOpenGLGraphicsRequirementsKHR(XrInstance instance, XrSystemId systemId, XrGraphicsRequirementsOpenGLKHR* graphicsRequirements)",
  "XrResult xrGetInstanceProcAddr(XrInstance instance, const void * name, IntPtr * fn)",
  "XrResult xrPollEvent(XrInstance instance, XrEventDataBuffer* eventData)",
  "XrResult xrBeginSession(XrSession session, const XrSessionBeginInfo * beginInfo)",
  "XrResult xrEndSession(XrSession session)",
  "XrResult xrCreateSwapchain(XrSession session, const XrSwapchainCreateInfo* createInfo, XrSwapchain* swapchain)",

  "XrResult xrEnumerateSwapchainFormats(XrSession session, uint formatCapacityInput, uint * formatCountOutput, long[] formats)",

  "XrResult xrEnumerateViewConfigurations(XrInstance instance, XrSystemId systemId, uint viewConfigurationTypeCapacityInput, uint * viewConfigurationTypeCountOutput, IntPtr viewConfigurationTypes)",
  "XrResult xrEnumerateViewConfigurations(XrInstance instance, XrSystemId systemId, uint viewConfigurationTypeCapacityInput, uint * viewConfigurationTypeCountOutput, XrViewConfigurationType[] viewConfigurationTypes)",

  "XrResult xrEnumerateViewConfigurationViews(XrInstance instance, XrSystemId systemId, XrViewConfigurationType viewConfigurationType, uint viewCapacityInput, uint* viewCountOutput, ref XrViewConfigurationView[] views)",
  "XrResult xrEnumerateViewConfigurationViews(XrInstance instance, XrSystemId systemId, XrViewConfigurationType viewConfigurationType, uint viewCapacityInput, uint* viewCountOutput, IntPtr views)",
  
  "XrResult xrEnumerateSwapchainImages(XrSwapchain swapchain, uint imageCapacityInput, uint* imageCountOutput, XrSwapchainImageBaseHeader* images)",
  "XrResult xrEnumerateSwapchainImages(XrSwapchain swapchain, uint imageCapacityInput, uint* imageCountOutput, IntPtr images)",

  "XrResult xrWaitFrame(XrSession session, const XrFrameWaitInfo * frameWaitInfo, XrFrameState * frameState)",
  "XrResult xrBeginFrame(XrSession session, const XrFrameBeginInfo* frameBeginInfo)",
  "XrResult xrEndFrame(XrSession session, const XrFrameEndInfo* frameEndInfo)",

  "XrResult xrCreateReferenceSpace(XrSession session, const XrReferenceSpaceCreateInfo* createInfo, XrSpace* space)",

  "XrResult xrEnumerateInstanceExtensionProperties(IntPtr layerName, uint32_t propertyCapacityInput, uint32_t* propertyCountOutput, XrExtensionProperties* properties)",
  "XrResult xrEnumerateInstanceExtensionProperties(IntPtr layerName, uint32_t propertyCapacityInput, uint32_t* propertyCountOutput, IntPtr properties)",

  "XrResult xrAcquireSwapchainImage(XrSwapchain swapchain, const XrSwapchainImageAcquireInfo* acquireInfo, uint32_t* index)",
  "XrResult xrWaitSwapchainImage(XrSwapchain swapchain, const XrSwapchainImageWaitInfo* waitInfo)",

  "XrResult xrReleaseSwapchainImage(XrSwapchain swapchain, const XrSwapchainImageReleaseInfo* releaseInfo)",

  "XrResult xrLocateViews(XrSession session, const XrViewLocateInfo* viewLocateInfo, XrViewState* viewState, uint32_t viewCapacityInput, uint32_t* viewCountOutput, XrView* views)",
  "XrResult xrLocateViews(XrSession session, const XrViewLocateInfo* viewLocateInfo, XrViewState* viewState, uint32_t viewCapacityInput, uint32_t* viewCountOutput, IntPtr views)",
  
  "XrResult xrLocateSpace(XrSpace space, XrSpace baseSpace, XrTime time, XrSpaceLocation* location)",
  
  "XrResult xrEnumerateEnvironmentBlendModes(XrInstance instance, XrSystemId systemId, XrViewConfigurationType viewConfigurationType, uint32_t environmentBlendModeCapacityInput, uint32_t* environmentBlendModeCountOutput, IntPtr environmentBlendModes)",
]

In [5]:
# Generate dll
final_out = ""
for fn in function_list:
  tokens = tokenize_code(fn)
  return_type = tokens[0]["string"]
  fn_name = tokens[1]["string"]
  args = []

  ctype = ""
  is_ref = False
  is_const = False
  is_array = False
  is_out = False
  for op in tokens[3:]:
    if op["string"] == ",":
      continue
    if op["string"] == "[":
      is_array = True
      continue
    if op["string"] == "]":
      is_array = True
      continue
    if op["string"] == ")":
      break
    if op["string"] == "*" or op["string"] == "**":
      is_ref = True
      continue
    if op["string"] == "const":
      is_const = True
      continue
    if op["string"] == "out":
      is_out = True
      continue
    if op["string"] == "ref":
      is_ref = True
      continue
    if ctype == "":
      ctype += op["string"]
    else:
      # Replace types
      if (ctype == "void" or ctype == "GLchar") and is_ref:
        is_ref = False
        ctype = "IntPtr"
      if ctype == "uint32_t":
        ctype = "uint"
      args.append({
        "type": ctype,
        "name": op["string"],
        "is_ref": is_ref,
        "is_const": is_const,
        "is_array": is_array,
        "is_out": is_out,
      })
      ctype = ""
      is_ref = False
      is_const = False
      is_array = False
      is_out = False
  
  # Fn name
  out = ""
  out += f"[DllImport(\"{lib_path}\", CharSet = CharSet.Ansi, CallingConvention = CallingConvention.Cdecl, SetLastError = true)]\n"
  out += f"public static extern {return_type} {fn_name}("
  for i in range(0, len(args)):
    if args[i]["is_ref"]:
      out += "ref "
    if args[i]["is_out"]:
      out += "out "
    out += args[i]["type"] + " "
    if args[i]["is_array"]:
      out += "[] "
    out += args[i]["name"]
    if (i < len(args) - 1):
      out += ", "
  out += f");\n\n"
  final_out += out

with open("./MegaLib/src/OS/Api/OpenXR_AutoGen.cs", "w") as file:
  file_data = """
using System;
using System.Runtime.InteropServices;
using XrFlags64 = System.UInt64;
using XrInstanceCreateFlags = System.UInt64;
using XrVersion = System.UInt64;
using XrInstance = System.IntPtr;
using XrSession = System.IntPtr;
using XrSpace = System.IntPtr;
using XrSwapchain = System.IntPtr;
using XrSystemId = System.UInt64;
using XrTime = System.Int64;
using XrBool32 = System.UInt32;
namespace MegaLib.OS.Api
  {
  """
  file_data += "public static partial class OpenXR {\n"
  file_data += final_out
  file_data += "}\n}\n"

  file.write(file_data)